### 1장 : 들어가기

- [페이스북의 국제 이주 패턴 찾기](https://www.facebook.com/notes/facebook-data-science/coordinated-migration/10151930946453859)
- [페이스북의 미식 축구팀 팬들의 주요 거주 지역](http://on.fb.me/1EQTvnO)
- 미국의 슈퍼마켓 체인점 Target은 구매 내역 데이터로 어떤 고객이 임신했는지 예측하여 아기 용품을 광고.
- 오바마 대선 캠프는 관심을 요하는 유권자들을 걸러 최적의 선거 모금 프로그램과 분위기를 선택 + 선거 참여율이 높아질 경우 당선에 도움이 되는 지역 찾음.

In [2]:
users = [
    {"id":0, "name":"Hero"},
    {"id":1, "name":"Dunn"},
    {"id":2, "name":"Sue"},
    {"id":3, "name":"Chi"},
    {"id":4, "name":"Thor"},
    {"id":5, "name":"Clive"},
    {"id":6, "name":"Hicks"},
    {"id":7, "name":"Devin"},
    {"id":8, "name":"Kate"},
    {"id":9, "name":"Klein"}
]
friendships = [(0,1), (0,2), (1,2), (1,3), (2,3), (3,4), (4,5), (5,6), (5,7), (6,8), (7,8), (8,9)]

In [10]:
# 친구 입력하기
for user in users:
    user["friends"] = []

for i,j in friendships :
    users[i]["friends"].append(users[j]["id"])
    users[j]["friends"].append(users[i]["id"])

In [11]:
users

[{'friends': [1, 2], 'id': 0, 'name': 'Hero'},
 {'friends': [0, 2, 3], 'id': 1, 'name': 'Dunn'},
 {'friends': [0, 1, 3], 'id': 2, 'name': 'Sue'},
 {'friends': [1, 2, 4], 'id': 3, 'name': 'Chi'},
 {'friends': [3, 5], 'id': 4, 'name': 'Thor'},
 {'friends': [4, 6, 7], 'id': 5, 'name': 'Clive'},
 {'friends': [5, 8], 'id': 6, 'name': 'Hicks'},
 {'friends': [5, 8], 'id': 7, 'name': 'Devin'},
 {'friends': [6, 7, 9], 'id': 8, 'name': 'Kate'},
 {'friends': [8], 'id': 9, 'name': 'Klein'}]

In [12]:
# 평균 친구 수 구하기
def mean_of_friends(users) :
    m = sum(len(user["friends"]) for user in users)/len(users)
    return m

In [13]:
mean_of_friends(users)

2.4

In [14]:
# 친구가 가장 많은/적은 사람
number_of_friends = [(user["name"], len(user["friends"])) for user in users]
number_of_friends

[('Hero', 2),
 ('Dunn', 3),
 ('Sue', 3),
 ('Chi', 3),
 ('Thor', 2),
 ('Clive', 3),
 ('Hicks', 2),
 ('Devin', 2),
 ('Kate', 3),
 ('Klein', 1)]

In [15]:
sorted_number_of_friends = sorted(number_of_friends, key=lambda user: user[1], reverse=True)
sorted_number_of_friends

[('Dunn', 3),
 ('Sue', 3),
 ('Chi', 3),
 ('Clive', 3),
 ('Kate', 3),
 ('Hero', 2),
 ('Thor', 2),
 ('Hicks', 2),
 ('Devin', 2),
 ('Klein', 1)]

- 친구 수가 3인 친구들이 네트워크에서 연결 중심성이 높은 것처럼 보이지만 네트워크를 그려보면 id=4인 Thor가 가운데에 위치하여 더 중심적인 역할을 하는 것처럼 보인다.
- 네트워크 중심성을 파악하면 페이스북같은 플랫폼에서 유용하게 사용될 듯.
- 네트워크 분석은 [21장](https://github.com/jsj267/Data-Science-from-Scratch/blob/master/21%EC%9E%A5.ipynb)참고

In [16]:
friends_dict = {}
for user in users:
    friends_dict.update({ user["id"] : user["friends"]})
friends_dict

{0: [1, 2],
 1: [0, 2, 3],
 2: [0, 1, 3],
 3: [1, 2, 4],
 4: [3, 5],
 5: [4, 6, 7],
 6: [5, 8],
 7: [5, 8],
 8: [6, 7, 9],
 9: [8]}

In [48]:
# 친구의 친구 찾기
def friends_of_friend_ids_bad(user, friends_dict):
    foaf_list = [foaf
                        for friend in user["friends"]
                        for foaf in friends_dict[friend]
                        if foaf != user["id"] and foaf not in user["friends"]]
    return list(set(foaf_list))

In [49]:
for user in users:
    print("user :", user["id"], "            foaf :", friends_of_friend_ids_bad(user, friends_dict))

user : 0             foaf : [3]
user : 1             foaf : [4]
user : 2             foaf : [4]
user : 3             foaf : [0, 5]
user : 4             foaf : [1, 2, 6, 7]
user : 5             foaf : [8, 3]
user : 6             foaf : [9, 4, 7]
user : 7             foaf : [9, 4, 6]
user : 8             foaf : [5]
user : 9             foaf : [6, 7]


In [59]:
# 서로가 함께 아는 친구(단, 사용자가 이미 아는 친구는 제외)
from collections import Counter

def friends_of_friend_ids(user, friends_dict):
    return Counter(foaf
                                for friend in user["friends"]
                                for foaf in friends_dict[friend]
                                if user["id"] != foaf and foaf in friends_dict[friend] and foaf not in user["friends"])
## collections.Counter() : 각 요소의 개수를 dict형태로 return(히스토그램이나 특정 문서에서 단어의 개수를 셀 때 유용.)

In [60]:
friends_of_friend_ids(users[3], friends_dict)

Counter({0: 2, 5: 1})

In [61]:
# 사용자의 (id, interest)
interests = [
    (0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"),
    (0, "Spark"), (0, "Storm"), (0, "Cassandra"),
    (1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
    (1, "Postgres"), (2, "Python"), (2, "scikit-learn"), (2, "scipy"),
    (2, "numpy"), (2, "statsmodels"), (2, "pandas"), (3, "R"), (3, "Python"),
    (3, "statistics"), (3, "regression"), (3, "probability"),
    (4, "machine learning"), (4, "regression"), (4, "decision trees"),
    (4, "libsvm"), (5, "Python"), (5, "R"), (5, "Java"), (5, "C++"),
    (5, "Haskell"), (5, "programming languages"), (6, "statistics"),
    (6, "probability"), (6, "mathematics"), (6, "theory"),
    (7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"),
    (7, "neural networks"), (8, "neural networks"), (8, "deep learning"),
    (8, "Big Data"), (8, "artificial intelligence"), (9, "Hadoop"),
    (9, "Java"), (9, "MapReduce"), (9, "Big Data")
]

In [62]:
# 특정 관심사를 공유하는 사용자
def data_scientists_who_like(target_interest):
    return [user_id for user_id, interest in interests if interest == target_interest]

In [73]:
# 관심사를 key, 사용자를 value로 dict를 만들면
interest_list = [interest for user, interest in interests]
interest_set = set(interest_list)

user_ids_by_interest = {}
for interest in interest_set :
    user_ids_by_interest[interest] = data_scientists_who_like(interest)
    
user_ids_by_interest

{'Big Data': [0, 8, 9],
 'C++': [5],
 'Cassandra': [0, 1],
 'HBase': [0, 1],
 'Hadoop': [0, 9],
 'Haskell': [5],
 'Java': [0, 5, 9],
 'Mahout': [7],
 'MapReduce': [9],
 'MongoDB': [1],
 'NoSQL': [1],
 'Postgres': [1],
 'Python': [2, 3, 5],
 'R': [3, 5],
 'Spark': [0],
 'Storm': [0],
 'artificial intelligence': [8],
 'decision trees': [4],
 'deep learning': [8],
 'libsvm': [4],
 'machine learning': [4, 7],
 'mathematics': [6],
 'neural networks': [7, 8],
 'numpy': [2],
 'pandas': [2],
 'probability': [3, 6],
 'programming languages': [5],
 'regression': [3, 4],
 'scikit-learn': [2, 7],
 'scipy': [2],
 'statistics': [3, 6],
 'statsmodels': [2],
 'theory': [6]}

In [78]:
# 교재에서는 이렇게
from collections import defaultdict
## defaultdict(list/int/set) : value를 각 자료형을 초기화해줌. int의 경우에는 default=0

user_ids_by_interest = defaultdict(list)

for user_id, interest in interests:
    user_ids_by_interest[interest].append(user_id)
    
user_ids_by_interest

defaultdict(list,
            {'Big Data': [0, 8, 9],
             'C++': [5],
             'Cassandra': [0, 1],
             'HBase': [0, 1],
             'Hadoop': [0, 9],
             'Haskell': [5],
             'Java': [0, 5, 9],
             'Mahout': [7],
             'MapReduce': [9],
             'MongoDB': [1],
             'NoSQL': [1],
             'Postgres': [1],
             'Python': [2, 3, 5],
             'R': [3, 5],
             'Spark': [0],
             'Storm': [0],
             'artificial intelligence': [8],
             'decision trees': [4],
             'deep learning': [8],
             'libsvm': [4],
             'machine learning': [4, 7],
             'mathematics': [6],
             'neural networks': [7, 8],
             'numpy': [2],
             'pandas': [2],
             'probability': [3, 6],
             'programming languages': [5],
             'regression': [3, 4],
             'scikit-learn': [2, 7],
             'scipy': [2],
             's

In [79]:
# 반대로 각 사용자별 interest
interest_by_user_id = defaultdict(list)

for user_id, interest in interests:
    interest_by_user_id[user_id].append(interest)

interest_by_user_id

defaultdict(list,
            {0: ['Hadoop',
              'Big Data',
              'HBase',
              'Java',
              'Spark',
              'Storm',
              'Cassandra'],
             1: ['NoSQL', 'MongoDB', 'Cassandra', 'HBase', 'Postgres'],
             2: ['Python',
              'scikit-learn',
              'scipy',
              'numpy',
              'statsmodels',
              'pandas'],
             3: ['R', 'Python', 'statistics', 'regression', 'probability'],
             4: ['machine learning', 'regression', 'decision trees', 'libsvm'],
             5: ['Python',
              'R',
              'Java',
              'C++',
              'Haskell',
              'programming languages'],
             6: ['statistics', 'probability', 'mathematics', 'theory'],
             7: ['machine learning',
              'scikit-learn',
              'Mahout',
              'neural networks'],
             8: ['neural networks',
              'deep learning',
       

In [82]:
# 특정사용자와 같은 interest를 가진 사용자. 
def most_common_interest_with(user):
    return  Counter(user_id 
                                    for interest in interest_by_user_id[user]
                                    for user_id in user_ids_by_interest[interest]
                                    if user_id != user)

In [84]:
most_common_interest_with(0)

Counter({1: 2, 5: 1, 8: 1, 9: 3})

In [85]:
# 연봉 / 근속기간
salaries_and_tenures = [(83000, 8.7), (88000, 8.1),
                        (48000, 0.7), (76000, 6),
                        (69000, 6.5), (76000, 7.5),
                        (60000, 2.5), (83000, 10),
                        (48000, 1.9), (63000, 4.2)]

In [86]:
# 근속연수에 따라 평균 연봉이 어떻게 달라지는가
salary_by_tenure = defaultdict(list)

for salary, tenure in salaries_and_tenures :
    salary_by_tenure[tenure].append(salary)

average_salary_by_tenure = {
    tenure : sum(salaries) / len(salaries)
    for tenure, salaries in salary_by_tenure.items()
}

average_salary_by_tenure

{0.7: 48000.0,
 1.9: 48000.0,
 2.5: 60000.0,
 4.2: 63000.0,
 6: 76000.0,
 6.5: 69000.0,
 7.5: 76000.0,
 8.1: 88000.0,
 8.7: 83000.0,
 10: 83000.0}

In [88]:
# 데이터 개수가 적어서 구간을 나눠보자
def tenure_bucket(tenure) :
    if tenure < 2:
        return "less than 2"
    elif tenure < 5 :
        return "between 2 and 5"
    else : 
        return "more than 5"

In [91]:
# 각 해당 bucket에 할당
salary_by_tenure_bucket = defaultdict(list)
for salary, tenure in salaries_and_tenures :
    bucket = tenure_bucket(tenure)
    salary_by_tenure_bucket[bucket].append(salary)

In [93]:
# 각 bucket의 평균 연봉
average_salary_by_bucket = {
    tenure_bucket : sum(salaries) / len(salaries)
    for tenure_bucket, salaries in salary_by_tenure_bucket.items()
}

average_salary_by_bucket

{'between 2 and 5': 61500.0,
 'less than 2': 48000.0,
 'more than 5': 79166.66666666667}

In [98]:
# 단어의 개수 세기(자연어처리)
words_and_counts = Counter(word 
                                                      for user, interest in interests
                                                      for word in interest.lower().split())

for word, count in words_and_counts.items():
    if count>1:
        print(word, count)

hbase 2
regression 2
networks 2
probability 2
scikit-learn 2
learning 3
statistics 2
hadoop 2
java 3
r 2
cassandra 2
neural 2
data 3
big 3
machine 2
python 3
